# Project Productie Proces
- Carlo Dieltjens - Raf Mesotten - Yannick Pottiez
- April 2024

## Stap 0: Importeren

#### Libraries

In [ ]:
import pandas as pd
import numpy as np
import os
import json
import sys
import random
import matplotlib.pyplot as plt 
import scipy.stats as stats
from scipy.stats import cauchy

In [ ]:
# Vermijd toekomstige deprecated-fouten bij onder meer histogrammen
pd.set_option('future.no_silent_downcasting', True)

#### Functies

In [ ]:
sys.path.append('../scripts')  # definieer path met scripts
from segment_calculations import create_segments
from simulate_production import simulate_production
from read_files import read_json_files
from make_a_chart import generate_point_distribution
from make_a_chart import generate_uniform_distribution
from make_a_chart import generate_normal_distribution
from make_a_chart import generate_cauchy_distribution
from make_a_chart import plot_segment_distributions
from make_a_chart import line_chart_daily_production

## Stap 1: Gegevens downloaden en organiseren

#### Definiëren van input paden
Alle input staat in de folder ```data\input```.

In [ ]:
input_path = os.path.join('..', 'data', 'input')

# Definieer paden voor BRU en STO
directory_path_BRU = os.path.join(input_path, 'data_productie', 'daily_production', 'BRU')
directory_path_STO = os.path.join(input_path, 'data_productie', 'daily_production', 'STO')

## Stap 2: Git Repository initialiseren
https://github.com/craftyraf/project-productie-proces

## Stap 3: Creëer een virtuele omgeving met Anaconda
Enkel de nodige pakketten werden geinstalleerd.
In README.md staat uitgelegd hoe je je virtual environment kan opzetten.

## Stap 4: Lezen van gegevens in Python

#### Inlezen van 1 .json bestand

In [ ]:
df = pd.read_json(os.path.join(input_path, 
                               'data_productie', 
                               'daily_production', 
                               'BRU', 
                               '20180101.json'), 
                  orient='index')
df

#### Inlezen van alle .json bestanden + samenvoegen in 1 dataframe voor alle locaties (met extra kolom: 'location')

In [ ]:
# Lees JSON-bestanden en maak DataFrame met 'Location' kolom
BRU_df = read_json_files(directory_path_BRU, 'BRU')
STO_df = read_json_files(directory_path_STO, 'STO')

# Concateneer BRU en STO DataFrames
combined_df = pd.concat([BRU_df, STO_df], ignore_index=True)

# Toon gecombineerde DataFrame
combined_df

#### Filter alle records waarbij onderhoud plaatsvond

In [ ]:
# Unieke waarden met hun aantallen voor 'maintenance' in BRU_df
unique_values_counts = combined_df['maintenance'].value_counts()
print(unique_values_counts)

# Behoud enkel de records met 'maintenance = No'
combined_df_no_maintenance = combined_df[combined_df['maintenance'] == 'No']

#### Controle of er NaN-waarden in het dataframe zit

Er zitten geen NaN-waarden in combined_df_no_maintenance:

In [ ]:
print("Aantal NaN-waarden per kolom in combined_df_no_maintenance:")
for column in combined_df_no_maintenance.columns:
    nan_count = combined_df_no_maintenance[column].isna().sum()
    print(f"{column}: {nan_count}")

#### Check datatypes en converteer naar het juiste format

In [ ]:
combined_df_no_maintenance.dtypes

In [ ]:
# Maak een nieuwe DataFrame met de schoongemaakte numerieke kolommen
combined_df_cleaned = combined_df_no_maintenance.copy()

# Converteer de kolommen naar numerieke datatypes
combined_df_cleaned['prod_loss'] = pd.to_numeric(combined_df_cleaned['prod_loss'], errors='coerce')
combined_df_cleaned['prod_loss_perc'] = pd.to_numeric(combined_df_cleaned['prod_loss_perc'], errors='coerce')
combined_df_cleaned['production'] = pd.to_numeric(combined_df_cleaned['production'], errors='coerce')

# Print de datatypes van de nieuwe DataFrame om te controleren
print(combined_df_cleaned.dtypes)

#### Creëer een histogram per locatie

In [ ]:
# Verkrijg de unieke locaties
unique_locations = combined_df_cleaned['location'].unique()

# Bereken het aantal rijen/kolommen benodigd voor de subplots
n_locs = len(unique_locations)
n_cols = 3  # Kies bijvoorbeeld 3 kolommen of een ander gewenst aantal
n_rows = n_locs // n_cols + (n_locs % n_cols > 0)

# Creëer een figuur en assen voor de subplots
fig, axs = plt.subplots(n_rows, n_cols, figsize=(5 * n_cols, 4 * n_rows), squeeze=False)

for i, loc in enumerate(unique_locations):
    # Bereken de huidige rij en kolom
    row = i // n_cols
    col = i % n_cols
    
    # Filter de data voor de huidige locatie
    loc_data = combined_df_cleaned[combined_df_cleaned['location'] == loc]['production']
    
    # Plot histogram van productie voor de huidige locatie
    axs[row, col].hist(loc_data, bins=50, alpha=0.7)
    axs[row, col].set_title(f'Histogram van productie {loc}')
    axs[row, col].set_xlabel('Productie')
    axs[row, col].set_ylabel('Frequentie')

# Verberg overbodige subplots als het aantal locaties niet perfect in de grid past
for i in range(n_locs, n_rows * n_cols):
    fig.delaxes(axs.flatten()[i])

# Toon de plot
plt.tight_layout()
plt.show()

### Stap 5: Bepaal een geschikte verdeling voor het dagelijkse productieproces

In bovenstaande grafieken is duidelijk zichtbaar dat een normaalverdeling niet zal volstaan om de data te fitten. Reden is de grote piek ter hoogte van 0, maar ook de lange linkerstaart samen met de korte rechterstaart. Om die reden besluiten we onze histogrammen (visueel) op te delen in 3 delen, en voor elk deel een geschikte fit te vinden.

#### BRU
In het geval van BRU kozen we voor een **puntverdeling** op 0, een **uniforme verdeling** voor het tussengedeelte, en een **normaalverdeling** voor de 'normale' productie.

In [ ]:
# Inputparameters BRU
location = 'BRU'
distribution_1 = generate_point_distribution
threshold_1 = 10**(-8)
distribution_2 = generate_uniform_distribution
threshold_2 = 750
distribution_3 = generate_normal_distribution

# Gebruik de functie 'create_segments' om de segmenten te maken (grenzen werden visueel bepaald)
# en het aandeel per segment te berekenen.
share_1, share_2, share_3, \
segment_1, segment_2, segment_3 = create_segments(location, threshold_1, threshold_2, combined_df_cleaned)

# Print shares (aandelen)
print(f"{location}: % dagen met [productie < {threshold_1}]:", share_1, '%')
print(f"{location}: % dagen met [{threshold_1} <= productie < {threshold_2}]:", share_2, '%')
print(f"{location}: % dagen met [productie >= {threshold_2}]:", share_3, '%')

# Genereer grafieken van de 3 segmenten
plot_segment_distributions(segment_1, distribution_1,
                           segment_2, distribution_2, 
                           segment_3, distribution_3,                           
                           location, 
                           threshold_1, 
                           threshold_2, 
                           combined_df_cleaned
                          )

#### STO
In het geval van STO kozen we voor een **puntverdeling** op 0, een **uniforme verdeling** voor het tussengedeelte, en een **cauchyverdeling** voor de 'normale' productie (heeft een scherpere piek dan de normaalverdeling).

In [ ]:
# Inputparameters STO
location = 'STO'
distribution_1 = generate_point_distribution
threshold_1 = 10**(-8)
distribution_2 = generate_uniform_distribution
threshold_2 = 150
distribution_3 = generate_cauchy_distribution

# Gebruik de functie 'create_segments' om de segmenten te maken (grenzen werden visueel bepaald)
# en het aandeel per segment te berekenen.
share_1, share_2, share_3, \
segment_1, segment_2, segment_3 = create_segments(location, threshold_1, threshold_2, combined_df_cleaned)

# Print shares (aandelen)
print(f"{location}: % dagen met [productie < {threshold_1}]:", share_1, '%')
print(f"{location}: % dagen met [{threshold_1} <= productie < {threshold_2}]:", share_2, '%')
print(f"{location}: % dagen met [productie >= {threshold_2}]:", share_3, '%')

# Genereer grafieken van de 3 segmenten
plot_segment_distributions(segment_1, distribution_1,
                           segment_2, distribution_2, 
                           segment_3, distribution_3,                           
                           location, 
                           threshold_1, 
                           threshold_2, 
                           combined_df_cleaned
                          )

### Stap 6: Simulatieprogramma ontwikkelen

In [ ]:
# Inputparameters BRU
location = 'BRU'
distribution_1 = generate_point_distribution
threshold_1 = 10**(-8)
distribution_2 = generate_uniform_distribution
threshold_2 = 750
distribution_3 = generate_normal_distribution

# Gebruik de functie 'create_segments' om de segmenten te maken (grenzen werden visueel bepaald)
# en het aandeel per segment te berekenen.
share_1, share_2, share_3, \
segment_1, segment_2, segment_3 = create_segments(location, threshold_1, threshold_2, combined_df_cleaned)

# Parameters van de verdelingen uitpakken
xmin, xmax = generate_uniform_distribution(segment_2, location, threshold_1, threshold_2
mean_norm, std_norm = generate_normal_distribution

def generate_random_value(share_1, share_2, distribution_2, xmin, xmax):
    """
    Generate a random value based on given shares.

    Parameters:
        share_1 (float): The first share.
        share_2 (float): The second share.
        distribution_2 (tuple): The range of the uniform distribution for the second case.
        xmin (float): The minimum value.
        xmax (float): The maximum value.

    Returns:
        int: The generated value.
    """
    random_number = random.random()  # Generate a random number between 0 and 1
    
    # Check conditions and return corresponding value
    if random_number < share_1:
        return 0
    elif random_number < share_2:
        # Generate a random number from the uniform distribution
        return random.uniform(xmin, xmax)
    else:
        return 20

generate_random_value(share_1, share_2, (xmin, xmax), xmin, xmax)
print(value)

In [ ]:
# Vraag de gebruiker om de duur van de simulatie (n dagen)
n = int(input("Voer het aantal dagen in voor de simulatie: "))

# Verkrijg de unieke locaties
unique_locations = combined_df_cleaned['location'].unique()

for i, loc in enumerate(unique_locations):  
    # Filter de data voor de huidige locatie
    location = unique_locations[i]
    loc_data = combined_df_cleaned[combined_df_cleaned['location'] == location]['production']
    
    # Parameters voor de normaalverdeling (gemiddelde en standaarddeviatie)
    mean, std = stats.norm.fit(loc_data) #data_combined_df_cleaned)

    # Simuleer de productie
    simulated_data = simulate_production(n, mean, std)

    # Toon de gesimuleerde productiewaarden
    print("Gesimuleerde productiewaarden:")
    for i, value in enumerate(simulated_data, 1):
        print(f"Dag {i}: {value}")

    # Maak een chart
    chart_title = "Gesimuleerde productiewaarden"
    line_chart_daily_production(simulated_data, location, chart_title)


# 7 Gebruik je simulatie